In [ ]:
#Para mostrar sugerencias con TAB
!pip install ipython
%config Completer.use_jedi = False

In [ ]:
#Para autocompletar
!pip install jupyter_contrib_nbextensions

In [ ]:
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter contrib nbextensions install --user

In [ ]:
#Se inicia el findspark
import findspark
findspark.init()

#Se configura el spark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Pyspark")
sc = SparkContext(conf = conf)

In [ ]:
def parseLines(x):
  x = x.split('::')[1:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[1] = float(x[1])
  return(tuple(x)) # Para par clave por lo general se representa en tuplas

  #(reduceByKey)hace la operación en las claves (películas) que son iguales
movilens = sc.textFile("./ml-1m/ratings.dat")

ratings = movilens.map(parseLines)\
                  .mapValues(lambda x: (x,1))\
                  .reduceByKey(lambda x , y: (x[0] + y[0], x[1] + y[1]))\
                  .mapValues(lambda x: x[0]/x[1])\
                  .sortBy(lambda x: -x[1])
ratings.collect()

[('787', 5.0),
 ('1830', 5.0),
 ('3607', 5.0),
 ('3280', 5.0),
 ('989', 5.0),
 ('3881', 5.0),
 ('3233', 5.0),
 ('3172', 5.0),
 ('3656', 5.0),
 ('3382', 5.0),
 ('3245', 4.8),
 ('53', 4.75),
 ('2503', 4.666666666666667),
 ('2905', 4.608695652173913),
 ('2019', 4.560509554140127),
 ('318', 4.554557700942973),
 ('858', 4.524966261808367),
 ('745', 4.52054794520548),
 ('50', 4.517106001121705),
 ('527', 4.510416666666667),
 ('1148', 4.507936507936508),
 ('1795', 4.5),
 ('3517', 4.5),
 ('557', 4.5),
 ('2480', 4.5),
 ('2309', 4.5),
 ('578', 4.5),
 ('3888', 4.5),
 ('439', 4.5),
 ('922', 4.491489361702127),
 ('1198', 4.477724741447892),
 ('904', 4.476190476190476),
 ('1178', 4.473913043478261),
 ('260', 4.453694416583082),
 ('1212', 4.452083333333333),
 ('750', 4.4498902706656915),
 ('3338', 4.444444444444445),
 ('720', 4.426940639269406),
 ('1207', 4.425646551724138),
 ('3435', 4.415607985480944),
 ('912', 4.412822049131217),
 ('670', 4.410714285714286),
 ('2762', 4.406262708418057),
 ('3030',

In [ ]:
def get_movies_info():
    #MovieID::Title::Genres
    movieNames = {}

    with open("./ml-1m/movies.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            movieNames[x[0]] = {"name" : x[1],
                                "genres": x[2].replace("\n", "")}
    return movieNames

In [ ]:
movieDictio = sc.broadcast(get_movies_info())
movieDictio.value["1"]

{'name': 'Toy Story (1995)', 'genres': "Animation|Children's|Comedy"}

In [ ]:
def get_users_info():
    #UserID::Gender::Age::Occupation::Zip-code
    usersInfo = {}

    with open("./ml-1m/users.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            usersInfo[x[0]] = {"gender" : x[1],
                               "Age" : x[2],
                                "Occupation": x[3]}
    return usersInfo

In [ ]:
userDictio = sc.broadcast(get_users_info())
userDictio.value

{'1': {'gender': 'F', 'Age': '1', 'Occupation': '10'},
 '2': {'gender': 'M', 'Age': '56', 'Occupation': '16'},
 '3': {'gender': 'M', 'Age': '25', 'Occupation': '15'},
 '4': {'gender': 'M', 'Age': '45', 'Occupation': '7'},
 '5': {'gender': 'M', 'Age': '25', 'Occupation': '20'},
 '6': {'gender': 'F', 'Age': '50', 'Occupation': '9'},
 '7': {'gender': 'M', 'Age': '35', 'Occupation': '1'},
 '8': {'gender': 'M', 'Age': '25', 'Occupation': '12'},
 '9': {'gender': 'M', 'Age': '25', 'Occupation': '17'},
 '10': {'gender': 'F', 'Age': '35', 'Occupation': '1'},
 '11': {'gender': 'F', 'Age': '25', 'Occupation': '1'},
 '12': {'gender': 'M', 'Age': '25', 'Occupation': '12'},
 '13': {'gender': 'M', 'Age': '45', 'Occupation': '1'},
 '14': {'gender': 'M', 'Age': '35', 'Occupation': '0'},
 '15': {'gender': 'M', 'Age': '25', 'Occupation': '7'},
 '16': {'gender': 'F', 'Age': '35', 'Occupation': '0'},
 '17': {'gender': 'M', 'Age': '50', 'Occupation': '1'},
 '18': {'gender': 'F', 'Age': '18', 'Occupation': '

In [ ]:
ratings.filter(lambda x: x[1] > 4.5)\
               .map(lambda x: (movieDictio.value[x[0]]["name"], x[1]))\
               .collect()

[('Gate of Heavenly Peace, The (1995)', 5.0),
 ('Follow the Bitch (1998)', 5.0),
 ('One Little Indian (1973)', 5.0),
 ('Baby, The (1973)', 5.0),
 ('Schlafes Bruder (Brother of Sleep) (1995)', 5.0),
 ('Bittersweet Motel (2000)', 5.0),
 ('Smashing Time (1967)', 5.0),
 ('Ulysses (Ulisse) (1954)', 5.0),
 ('Lured (1947)', 5.0),
 ('Song of Freedom (1936)', 5.0),
 ('I Am Cuba (Soy Cuba/Ya Kuba) (1964)', 4.8),
 ('Lamerica (1994)', 4.75),
 ('Apple, The (Sib) (1998)', 4.666666666666667),
 ('Sanjuro (1962)', 4.608695652173913),
 ('Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)',
  4.560509554140127),
 ('Shawshank Redemption, The (1994)', 4.554557700942973),
 ('Godfather, The (1972)', 4.524966261808367),
 ('Close Shave, A (1995)', 4.52054794520548),
 ('Usual Suspects, The (1995)', 4.517106001121705),
 ("Schindler's List (1993)", 4.510416666666667),
 ('Wrong Trousers, The (1993)', 4.507936507936508)]

In [ ]:
def query_edad(edad, genero_user, genero_peli):
    datos.map(lambda x: x.split("::"))\
        .filter(lambda x: userDistio)

In [ ]:
##Vamos a usar los diccionarios directamente

In [ ]:
#UserID::MovieID::Rating::Timestamp
movilens.first()

'1::1193::5::978300760'

In [ ]:
def parseLineN(x):
  x = x.split('::')[:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[2] = float(x[2])
  return(x) # Para par clave por lo general se representa en tuplas

movilens.map(parseLineN)

PythonRDD[14] at RDD at PythonRDD.scala:53

In [ ]:
Age = None
gender = None
Occupation = None
genres = None

if Age != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Age"] == Age)

if gender != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["gender"] == gender)

if Occupation != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Occupation"] == Occupation)


movilens.map(lambda x: x.split("::"))\
        .map(lambda x: (userDictio.value[x[0]]["Age"],movieDictio.value[x[1]]["name"]))\
        .collect()

[('1', "One Flew Over the Cuckoo's Nest (1975)"),
 ('1', 'James and the Giant Peach (1996)'),
 ('1', 'My Fair Lady (1964)'),
 ('1', 'Erin Brockovich (2000)'),
 ('1', "Bug's Life, A (1998)"),
 ('1', 'Princess Bride, The (1987)'),
 ('1', 'Ben-Hur (1959)'),
 ('1', 'Christmas Story, A (1983)'),
 ('1', 'Snow White and the Seven Dwarfs (1937)'),
 ('1', 'Wizard of Oz, The (1939)'),
 ('1', 'Beauty and the Beast (1991)'),
 ('1', 'Gigi (1958)'),
 ('1', 'Miracle on 34th Street (1947)'),
 ('1', "Ferris Bueller's Day Off (1986)"),
 ('1', 'Sound of Music, The (1965)'),
 ('1', 'Airplane! (1980)'),
 ('1', 'Tarzan (1999)'),
 ('1', 'Bambi (1942)'),
 ('1', 'Awakenings (1990)'),
 ('1', 'Big (1988)'),
 ('1', 'Pleasantville (1998)'),
 ('1', 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 ('1', 'Back to the Future (1985)'),
 ('1', "Schindler's List (1993)"),
 ('1', 'Meet Joe Black (1998)'),
 ('1', 'Pocahontas (1995)'),
 ('1', 'E.T. the Extra-Terrestrial (1982)'),
 ('1', 'Titanic (1997)'),
 ('1', 

In [ ]:
sc.stop()